<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/preprocess_recipeQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Mounted at /content/gdrive


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


100% 2.54G/2.55G [01:29<00:00, 25.8MB/s]
100% 2.55G/2.55G [01:29<00:00, 30.4MB/s]
--2020-11-30 14:42:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-30 14:42:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-30 14:42:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 20

# New Section

In [143]:
import json
import numpy as np
import tensorflow as tf
import re
import string

def read_file(file_name):
  with open(file_name, encoding='utf-8') as f:
    data = json.load(f)['data']
  textual_cloze = [item for item in data if item["task"]=="textual_cloze"]
  visual_cloze = [item for item in data if item["task"]=="visual_cloze"]
  visual_coherence = [item for item in data if item["task"]=="visual_coherence"]
  visual_ordering = [item for item in data if item["task"]=="visual_ordering"]
  return textual_cloze, visual_cloze, visual_coherence, visual_ordering

In [ ]:
def tokenize(word_index)

In [132]:
def delete_keys(dataset):
  match = re.compile('[^A-Za-z]')
  s = ""
  if not dataset[0]["task"]=="textual_cloze":
    for data in dataset:
      for step in data['context']:
        step['title'] = re.sub(match,' ', step['title']).lower()
        step['body'] = re.sub(match,' ', step['body']).lower()
        s += step['title']+ step['body']
  else:
    for data in dataset:
      for step in data['context']:
        step['body'] = re.sub(match,' ', step['body']).lower()
        s += step['body']
      for i in range(len(data['choice_list'])):
        data['choice_list'][i] = re.sub(match,' ', data['choice_list'][i]).lower()
        s += data['choice_list'][i]
      for i in range(len(data['question'])):
        data['question'][i] = re.sub(match,' ', data['question'][i]).lower()
        s += data['question'][i]
  if (dataset[0]["task"]=="visual_coherence"):
    [data.pop("question") for data in dataset]
  if (dataset[0]["task"]=="visual_ordering"):
    [data.pop("question") for data in dataset]
    for data in dataset:
      order = data["choice_list"][0]
      data["image_list"] = {0:order[0], 1:order[1], 2:order[2], 3:order[3]}
      img2ind = {v:k for k,v in data["image_list"].items()}
      data["choice_list"]=[[img2ind[img] for img in choice] for choice in data["choice_list"]]
  [[data.pop(key,None) for key in ["context_modality", "split", "qid", "question_modality", "task", "question_text"]] for data in dataset]
  [[[step.pop(key) for key in ["id","videos"]] for step in data['context']] for data in dataset]
  
#  print(json.dumps(dataset[0], indent=4))

In [38]:
def load_image(file_path):
# Load image
  image = tf.io.decode_jpeg(tf.io.read_file(file_path),channels=3)
# Convert image to normalized float [0, 1]
  image = tf.image.convert_image_dtype(image,tf.float32)
# resize image
  image = tf.image.resize(image, [256,256])
# Rescale data to range (-1, 1)
  image = (image - 0.5) * 2
  return image

In [75]:
from copy import deepcopy
def data_iter(batch_size, dataset, task, split):
  num_input = len(dataset)
  # np.random.shuffle(dataset)
  for i in range(num_input // batch_size):
    Xs = deepcopy(dataset[i*batch_size:(i+1)*batch_size])
    Ys = [item.pop("answer") for item in Xs]
    if task=="textual_cloze":
      for X in Xs:
        for step in X["context"]:
          step["images"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in step["images"]]
    if task=="visual_cloze":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
        X["question"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) if not item=="@placeholder" else "@placeholder"  for item in X["question"]]
    if task=="visual_coherence":
      for X in Xs:
        X["choice_list"] = [load_image("data/images/images-qa/"+split+"/images-qa/"+item) for item in X["choice_list"]]
    if task=="visual_ordering":
      for X in Xs:
        for k,v in X["image_list"].items():
          X["image_list"][k] = load_image("data/images/images-qa/"+split+"/images-qa/"+X["image_list"][k])
    yield Xs, Ys



In [40]:
def load_embeddings(path_to_glove_file = '/content/glove.6B.100d.txt'):
  embedding_index = {}
  with open(path_to_glove_file) as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      embedding_index[word] = coefs
  return embedding_index

def get_embedding_layer(voc, word_index, embeddings_index, embedding_dim = 100):
  num_tokens = len(word_index) + 2
  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
  embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
  )
  return embedding_layer

In [41]:
def preprocess(batch_size, split):
  textual_cloze, visual_cloze, visual_coherence, visual_ordering=read_file("data/"+split+" recipeqa.json")
  delete_keys(textual_cloze)
  delete_keys(visual_cloze)
  delete_keys(visual_coherence)
  delete_keys(visual_ordering)
  textual_cloze_iter = data_iter(batch_size, textual_cloze, "textual_cloze", split)
  visual_cloze_iter = data_iter(batch_size, visual_cloze, "visual_cloze", split)
  visual_coherence_iter = data_iter(batch_size, visual_coherence, "visual_coherence", split)
  visual_ordering_iter = data_iter(batch_size, visual_ordering, "visual_ordering", split)
  return textual_cloze_iter, visual_cloze_iter, visual_coherence_iter, visual_ordering_iter

In [178]:
def main():
  batch_size = 50
  train_it1, train_it2, train_it3, train_it4 = preprocess(batch_size, "train")
  dic = {'unk':0, 'pad':1}
  # for i in train_it1:
  #   print(i['context'])
  #   for batch in i
  #   for j in i['context']:
  #     j['body']=j['body'].split()
  #     j['body']=[k for k in j['body'] if k!='']
  #   for j in i['choice_list']:
  #     j['body']=[k for k in j if k!='']
  #   for j in i['question']:
  #     j['body']=[k for k in j if k!='']
  
  # word_index = words_to_index([train_it1, train_it2, train_it3, train_it4])
  test_it1, test_it2, test_it3, test_it4 = preprocess(batch_size, "test")
  val_it1, val_it2, val_it3, val_it4 = preprocess(batch_size, "val")
  # Usage:
  for Xs, Ys in train_it1:
    print(json.dumps(Xs[0], indent=4, default=lambda x:"tf_tensor"))
    print(Ys[0])
    break

In [179]:
if __name__=='__main__':
  main()

TypeError: ignored

In [156]:
train_it1, train_it2, train_it3, train_it4 = preprocess(10, "train")

In [169]:
for x,y in train_it1:
  # print(x[0]['context'],y[0])
  for i in x[0]['context']:
    print(i['body'])

  for i in x[0]['choice_list']:
    print(i)
  break


bacon bacon bacon salt  table and or sea salt  pepper mortar and pestle food processor containers to hold finished product


KeyError: ignored